## Declare Required Libraries

In [ ]:
import math
from datetime import datetime
from datetime import date
import csv
import pandas as pd
import folium

## Set Filename to Analize

In [ ]:
name = 'gps.txt'

## Load NMEA file

In [ ]:
names = []
for i in range(24):
    names.append('COL' + str(i+1))
nmeaDF = pd.read_csv(name, header=None, names=names, comment='#',dtype='object')
nmeaDF = nmeaDF.query("not COL1.str.contains('^.+program.+$', regex=True)")

## Check Message Types in file

In [ ]:
nmeaDF['COL1'].unique()

## Analyzer Definition

In [ ]:
def to_cksum(s):
    arr = s.split('*')
    if (len(arr)>1):
        return arr[0], arr[1]
    else:
        return arr[0], None

def to_time(value):
   if isinstance(value, float) and math.isnan(value):
       return None
   s = str(value)
   if '.' in s:
        return datetime.strptime(s, '%H%M%S.%f').time()
   else:
        return datetime.strptime(s, '%H%M%S').time()

def to_date(str):
   return datetime.strptime(str, '%d%m%y').date()

def to_datetime(dstr, tstr):
    if '.' in tstr:
        return datetime.strptime(dstr + ' ' + tstr, '%d%m%y %H%M%S.%f')
    else:
        return datetime.strptime(dstr + ' ' + tstr, '%d%m%y %H%M%S')

def to_degree(dms, flag):
    d = math.floor(dms/100) + 100*(dms/100 - math.floor(dms/100))/60
    if flag == 'S' or flag == 'W':
        return -1 * d
    else:
        return d
    
def to_dir_diff(diff, flag):
    if diff == '':
        return None
    
    d = float(diff)
    if flag == 'W':
        return -1 * d
    else:
        return d

In [ ]:
class NMEAParser:
    def parse_cksum(self, row):
        return None
    
    def parse_status(self, row):
        return None
    
    def parse_mode(self, row):
        return None
    
    def parse_time(self, row):
        return None
    
    def parse_date(self, row):
        return None
    
    def parse_datetime(self, row):
        return None
    
    def parse_tz_offset(self, row):
        return None
    
    def parse_latitude(self, row):
        return None
    
    def parse_longitude(self, row):
        return None
    
    def parse_knot(self, row):
        return None
    
    def parse_kmh(self, row):
        return None
    
    def parse_true_azimuth(self, row):
        return None
    
    def parse_declination(self, row):
        return None
    
    def parse_satellites(self, now):
        return None
    
    def parse_altitude(self, row):
        return None
    
    def parse_geoid(self, row):
        return None
    
    def parse_age(self, row):
        return None
    
    def parse_magnetic_azimuth(self, row):
        return None
    
    def parse_sv_type(self, row):
        return None
    
    def parse_sv_msgs(self, row):
        return None
    
    def parse_sv_msg_no(self, row):
        return None
    
    def parse_sv_visibles(self, row):
        return None
    
    def parse_hdop(self, row):
        return None
    
    def parse_vdop(self, row):
        return None
       
    def parse_pdop(self, row):
        return None
    
    def parse_sat_type(self, row):
        return None
    
    def parse_sat_nos(self, row):
        return None
    
    def parse_msg_num(self, row):
        return None
    
    def parse_msg_no(self, row):
        return None
    
    def parse_msg_type(self, row):
        return None
    
    def parse_msg_txt(self, row):
        return None

In [ ]:
class RMCParser(NMEAParser):

    def parse_status(self, row):
        return row['COL3']
        
    def parse_time(self, row):
        if self.parse_status(row) == 'A':
            return to_time(row['COL2'])
        else:
            return super().parse_time(row)
  
    def parse_datetime(self, row):
        if self.parse_status(row) == 'A':
            return to_datetime(row['COL10'], row['COL2'])
        
    def parse_latitude(self, row):
        if self.parse_status(row) == 'A':
            return to_degree(float(row['COL4']), row['COL5'])
        else:
            return super().parse_latitude(row)
        
    def parse_longitude(self, row):
        if self.parse_status(row) == 'A':
           return to_degree(float(row['COL6']), row['COL7'])
        else:
           return super().parse_longitude(row) 
     
    def parse_knot(self, row):
        if self.parse_status(row) == 'A':
            return float(row['COL8'])
        else:
            return super().parse_knot(row)

    def parse_true_azimuth(self, row):
        if self.parse_status(row) == 'A':
            return float(row['COL9']) 
        
    def parse_date(self, row):
        if self.parse_status(row) == 'A':
            return to_date(row['COL10'])
        else:
            return super().parse_date(row)
    
    def parse_declination(self, row):
        if self.parse_status(row) == 'A':
            return to_dir_diff(row['COL11'], row['COL12'])
        else:
            return super().parse_declination(row)

    def parse_mode(self, row):
        return to_cksum(row['COL13'])[0]
        
    def parse_cksum(self, row):
        return to_cksum(row['COL13'])[1]    

In [ ]:
class GGAParser(NMEAParser):
    def parse_cksum(self, row):
        return to_cksum(row['COL15'])[1]
    
    def parse_mode(self, row):
        return row['COL7']
    
    def parse_time(self, row):
        if self.parse_mode(row) != 0:
            return to_time(row['COL2'])
        else:
           return super().parse_time(row) 

    def parse_latitude(self, row):
        if self.parse_mode(row) != '0':
            return to_degree(float(row['COL3']), row['COL4'])
        else:
           return super().parse_latitude(row)  
    
    def parse_longitude(self, row):
        if self.parse_mode(row) != '0':
            return to_degree(float(row['COL5']), row['COL6'])
        else:
            return super().parse_longitude(row)  
        
    def parse_satellites(self, row):
        return row['COL8']
    
    def parse_hdop(self, row):
        if self.parse_mode(row) != '0':
            return float(row['COL9'])
        else:
            return super().parse_hdop(row)  
    
    def parse_altitude(self, row):
        if self.parse_mode(row) != '0':
            return float(row['COL10'])
        else:
            return super().parse_altitude(row)  
        
    def parse_geoid(self, row):
        if self.parse_mode(row) != '0':
            return float(row['COL12'])
        else:
            return super().parse_geoid(row)
        
    def parse_age(self, row):
        if self.parse_mode(row) == '2':
            if row['COL14'] !=  '':
                return float(row['COL14'])
        return super().parse_age(row)

In [ ]:
class ZDAParser(NMEAParser):
    def parse_cksum(self, row):
        return to_cksum(row['COL7'])[1]
    
    def parse_time(self, row):
        return to_time(row['COL2'])

    def parse_date(self, row):
        if isinstance(row['COL3'], float) and math.isnan(row['COL3']):
            return super().parse_datetime(row) 
        return datetime(int(row['COL5']), int(row['COL4']), int(row['COL3']))
    
    def parse_tz_offset(self, row):
        if isinstance(row['COL3'], float) and math.isnan(row['COL3']):
            return super().parse_tz_offset(row)
        hh = row['COL6']
        if len(hh) == 2:
            hh = '+' + hh
        mm = to_cksum(row['COL7'])[0]
        return hh + ':' + mm

            

In [ ]:
class VTGParser(NMEAParser):
    def parse_true_azimuth(self, row):
        if self.parse_mode(row) != 'M':
            return float(row['COL2'])
        else:
            return super().parse_true_azimuth(row)

    def parse_magnetic_azimuth(self, row):
        if row['COL4'] != '':
            return float(row['COL4'])
        else:
            return super().parse_magnetic_azimuth(row)

    def parse_mode(self, row):
        return row['COL10']
    
    def parse_knot(self, row):
        if self.parse_mode(row) != 'M':
            return float(row['COL6'])
        else:
            return super().parse_knot(row)
        
    def parse_kmh(self, row):
        if self.parse_mode(row) != 'M':
            return float(row['COL8'])
        else:
            return super().parse_kmh(row)
        
    def parse_mode(self, row):
        return to_cksum(row['COL10'])[0]
    
    def parse_cksum(self, row):
        return to_cksum(row['COL10'])[1]

In [ ]:
class GLLParser(NMEAParser):
    def parse_latitude(self, row):
        if self.parse_status(row) != 'V':
            return to_degree(float(row['COL2']), row['COL3'])
        else:
            return super().parse_latitude(row)
        
    def parse_longitude(self, row):
        if self.parse_status(row) != 'V':
           return to_degree(float(row['COL4']), row['COL5'])
        else:
           return super().parse_longitude(row) 
     
    def parse_time(self, row):
        if self.parse_status(row) != 'V':
            return to_time(row['COL6'])
        else:
            return super().parse_time(row)
        
    def parse_status(self, row):
        return row['COL7']
    
    def parse_mode(self, row):
        return to_cksum(row['COL8'])[0]
    
    def parse_cksum(self, row):
        return to_cksum(row['COL8'])[1]

In [ ]:
class GSVParser(NMEAParser):
    def parse_sv_type(self, row):
        rtype = row['COL1']
        return rtype[2]
    
    def parse_sv_msgs(self, row):
        if (row['COL2'] == 'A'):
            print(row['COL1'], row['COL2'], row['COL3'])
        return int(row['COL2'])
    
    def parse_sv_msg_no(self, row):
        return int(row['COL3'])
    
    def parse_sv_visibles(self, row):
        return to_cksum(row['COL4'])[0]
    
    def parse_cksum(self, row):
        for i in range(24):
            name = 'COL' + str(i+1)
            if '*' in str(row[name]):
                return to_cksum(row[name])[1]
        
        return super().parse_cksum(row)
    


In [ ]:
class GSAParser(NMEAParser):
    def parse_mode(self, row):
        return row['COL2']
    
    def parse_status(self, row):
        return row['COL3']

    def parse_sat_nos(self, row):
        sat_nos = []
        for i in range(12):
            name = 'COL' + str(i + 4)
            if row[name] != '':
                sat_nos.append(row[name])
        return sat_nos
    
    def parse_pdop(self, row):
        return float(row['COL16'])
    
    def parse_hdop(self, row):
        return float(row['COL17'])
        
    def parse_vdop(self, row):
        return to_cksum(row['COL18'])[0]

    def parse_sat_type(self, row):
        return None

    def parse_cksum(self, row):
        return to_cksum(row['COL18'])[1]

In [ ]:
class TXTParser(NMEAParser):
    def parse_msg_num(self, row):
        return row['COL2']
    
    def parse_msg_no(self, row):
        return row['COL3']
    
    def parse_msg_type(self, row):
        return row['COL4']
    
    def parse_msg_txt(self, row):
        return to_cksum(row['COL5'])[0]
    
    def parse_cksum(self, row):
        return to_cksum(row['COL5'])[1]

In [ ]:
class Parser:
    nmea = NMEAParser()
    rmc = RMCParser()
    gga = GGAParser()
    zda = ZDAParser()
    vtg = VTGParser()
    gll = GLLParser()
    gsv = GSVParser()
    gsa = GSAParser()
    txt = TXTParser()

    def select(row):
        rtype = row['COL1']
        if rtype == '$GNRMC' or rtype == '$GPRMC':
            return Parser.rmc
        elif rtype == '$GNGGA' or rtype == '$GPGGA':
            return Parser.gga
        elif rtype == '$GNZDA':
            return Parser.zda
        elif rtype == '$GNVTG':
            return Parser.vtg
        elif rtype == '$GNGLL' or rtype == '$GPGLL':
            return Parser.gll
        elif rtype == '$GPGSV' or rtype == '$GLGSV' or rtype == '$GBGSV' or rtype == '$GAGSV' or rtype == '$GQGSV' or rtype =='$BDGSV':
            return Parser.gsv
        elif rtype =='$GPGSA' or rtype == '$BDGSA':
            return Parser.gsa
        elif rtype == '$GPTXT':
            return Parser.txt
        else:
            return Parser.nmea

    def parse_talker_id(row):
        return row['COL1'][1:2]

    def parse_cksum(row):
        return Parser.select(row).parse_cksum(row)
    
    def parse_status(row):
        return Parser.select(row).parse_status(row)

    def parse_mode(row):
        return Parser.select(row).parse_mode(row)
    
    def parse_time(row):
        return Parser.select(row).parse_time(row)
    
    def parse_date(row):
        return Parser.select(row).parse_date(row)
    
    def parse_datetime(row):
        return Parser.select(row).parse_datetime(row)
    
    def parse_tz_offset(row):
        return Parser.select(row).parse_tz_offset(row)
    
    def parse_latitude(row):
        return Parser.select(row).parse_longitude(row)
    
    def parse_longitude(row):
        return Parser.select(row).parse_latitude(row)
    
    def parse_knot(row):
        return Parser.select(row).parse_knot(row)

    def parse_kmh(row):
        return Parser.select(row).parse_kmh(row)
        
    def parse_true_azimuth(row):
        return Parser.select(row).parse_true_azimuth(row)
    
    def parse_declination(row):
        return Parser.select(row).parse_declination(row)
    
    def parse_satellites(row):
        return Parser.select(row).parse_satellites(row)
    
    def parse_hdop(row):
        return Parser.select(row).parse_hdop(row)
    
    def parse_altitude(row):
        return Parser.select(row).parse_altitude(row)
    
    def parse_geoid(row):
        return Parser.select(row).parse_geoid(row)
    
    def parse_age(row):
        return Parser.select(row).parse_age(row)
    
    def parse_magnetic_azimuth(row):
        return Parser.select(row).parse_magnetic_azimuth(row)
    
    def parse_sv_type(row):
        return Parser.select(row).parse_sv_type(row)
    
    def parse_sv_msgs(row):
        return Parser.select(row).parse_sv_msgs(row)
    
    def parse_sv_msg_no(row):
        return Parser.select(row).parse_sv_msg_no(row)
    
    def parse_sv_visibles(row):
        return Parser.select(row).parse_sv_visibles(row)

    def parse_vdop(row):
        return Parser.select(row).parse_vdop(row)
       
    def parse_pdop(row):
        return Parser.select(row).parse_pdop(row)
    
    def parse_sat_type(row):
        return Parser.select(row).parse_sat_type(row)
    
    def parse_sat_nos(row):
        return Parser.select(row).parse_sat_nos(row)
    
    def parse_msg_num(row):
        return Parser.select(row).parse_msg_num(row)
    
    def parse_msg_no(row):
        return Parser.select(row).parse_msg_no(row)
    
    def parse_msg_type(row):
        return Parser.select(row).parse_msg_type(row)
    
    def parse_msg_txt(row):
        return Parser.select(row).parse_msg_txt(row)

## Analize NMEA messages

In [ ]:
nmeaDF['talker_id']   = nmeaDF.apply(Parser.parse_talker_id,    axis=1)
nmeaDF['cksum']       = nmeaDF.apply(Parser.parse_cksum,        axis=1)

nmeaDF['status']      = nmeaDF.apply(Parser.parse_status,       axis=1)
nmeaDF['mode']        = nmeaDF.apply(Parser.parse_mode,         axis=1)
nmeaDF['time']        = nmeaDF.apply(Parser.parse_time,         axis=1)
nmeaDF['date']        = nmeaDF.apply(Parser.parse_date,         axis=1)
nmeaDF['datetime']    = nmeaDF.apply(Parser.parse_datetime,     axis=1)
nmeaDF['tz_offset']   = nmeaDF.apply(Parser.parse_tz_offset,    axis=1)

nmeaDF['latitude']    = nmeaDF.apply(Parser.parse_latitude,     axis=1)
nmeaDF['longitude']   = nmeaDF.apply(Parser.parse_longitude,    axis=1)

nmeaDF['knot']        = nmeaDF.apply(Parser.parse_knot,         axis=1)
nmeaDF['kmh']         = nmeaDF.apply(Parser.parse_kmh,          axis=1)

nmeaDF['satellites']  = nmeaDF.apply(Parser.parse_satellites,   axis=1)
nmeaDF['hdop']        = nmeaDF.apply(Parser.parse_hdop,         axis=1)

nmeaDF['altitude']    = nmeaDF.apply(Parser.parse_altitude,     axis=1)
nmeaDF['geoid']       = nmeaDF.apply(Parser.parse_geoid,        axis=1)
nmeaDF['age']         = nmeaDF.apply(Parser.parse_age,          axis=1)

nmeaDF['true_azimuth']         = nmeaDF.apply(Parser.parse_true_azimuth,      axis=1)
nmeaDF['magnetic_azimuth']     = nmeaDF.apply(Parser.parse_magnetic_azimuth,  axis=1)
nmeaDF['magnetic_declination'] = nmeaDF.apply(Parser.parse_declination,       axis=1)

nmeaDF['sv_type']     = nmeaDF.apply(Parser.parse_sv_type,      axis=1)
nmeaDF['sv_msgs']     = nmeaDF.apply(Parser.parse_sv_msgs,      axis=1)
nmeaDF['sv_msg_no']   = nmeaDF.apply(Parser.parse_sv_msg_no,    axis=1)
nmeaDF['sv_vlsibles'] = nmeaDF.apply(Parser.parse_sv_visibles,  axis=1)

nmeaDF['pdop']        = nmeaDF.apply(Parser.parse_pdop,         axis=1)
nmeaDF['vdop']        = nmeaDF.apply(Parser.parse_vdop,         axis=1)
nmeaDF['sat_type']    = nmeaDF.apply(Parser.parse_sat_type,     axis=1)
nmeaDF['sat_nos']     = nmeaDF.apply(Parser.parse_sat_nos,      axis=1)

nmeaDF['msg_num']     = nmeaDF.apply(Parser.parse_msg_num,      axis=1)
nmeaDF['msg_no']      = nmeaDF.apply(Parser.parse_msg_no,       axis=1)
nmeaDF['msg_type']    = nmeaDF.apply(Parser.parse_msg_type,     axis=1)
nmeaDF['msg_txt']     = nmeaDF.apply(Parser.parse_msg_txt,      axis=1)

### Non analized message types

In [ ]:
nmeaDF.query("cksum.isnull()")['COL1'].unique()

### TXT messages

In [ ]:
nmeaDF.query("COL1.str.contains('^.+TXT$', regex=True)").loc[:, ['COL1','msg_num','msg_no','msg_type','msg_txt','cksum']]

### GSA Messages

In [ ]:
nmeaDF.query("COL1.str.contains('^.+GSA', regex=True)").loc[:,['COL1','mode','status','sat_nos','pdop','hdop','vdop','sat_type','cksum']]

### GSV Messages

In [ ]:
nmeaDF.query("COL1.str.contains('^.+GSV$', regex=True)").loc[:, ['COL1','sv_type','sv_msgs','sv_msg_no','sv_vlsibles','cksum']]

### GLL Messages

In [ ]:
nmeaDF.query("COL1.str.contains('^.+GLL$', regex=True)").loc[:, ['COL1','latitude','longitude','time','status','mode','cksum']]

### VTG Messages

In [ ]:
nmeaDF.query("COL1.str.contains('^.+VTG$', regex=True)").loc[:, ['COL1','true_azimuth','magnetic_azimuth','knot','kmh','mode','cksum']]

### ZDA Messages

In [ ]:
nmeaDF.query("COL1.str.contains('^.+ZDA$', regex=True)").loc[:, ['COL1','time','date','tz_offset','cksum']]

### GGA Messages

In [ ]:
nmeaDF.query("COL1.str.contains('^.+GGA$', regex=True)").loc[:, ['COL1','time','latitude','longitude','mode','satellites','hdop','altitude','geoid','age','cksum']]

### RMC Messages

In [ ]:
nmeaDF.query("COL1.str.contains('^.+RMC$', regex=True)").loc[:, ['COL1','datetime','status','latitude','longitude','knot','true_azimuth','magnetic_declination','mode','cksum']]

## Display Route from GPRMC Messages

In [ ]:
rmcDF = nmeaDF.query('COL1 == "$GPRMC" and COL3 =="A"')
locations = rmcDF[['longitude', 'latitude']].values.tolist()
locations
map = folium.Map(location=locations[0], tiles="OpenStreetMap", zoom_start=19)
folium.PolyLine(locations=locations).add_to(map)
map